In [5]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from colormath.color_objects import sRGBColor, XYZColor
from colormath.color_conversions import convert_color
%matplotlib

Using matplotlib backend: Qt5Agg


In [14]:
MF = np.genfromtxt("lin2012xyz2e_1_7sf.csv", delimiter=',') 
print(MF.shape)
CMF = MF[:, 1:4]

(441, 4)


In [15]:
def EdgeOptimal(i, delta, CMF):
    """
    Optimal windowing function without wrapping
    
    i: center wavelength-index
    delta: width of the window
    CMF: color matching function
    """
    Norm = np.sum(CMF, 0)
    c0 = 0
    c1 = 0
    c2 = 0
    for j in range(int(i-delta/2), int(i+delta/2), 1):
        if( (j > 0) and (j < CMF.shape[0]) ):
            c0 += CMF[j,0]
            c1 += CMF[j,1]
            c2 += CMF[j,2]
    return (c0/Norm[0], c1/Norm[1], c2/Norm[2])

def DoubleOptimal(i, delta, CMF):
    """
    Optimal windowing function with wrapping
    
    i: center wavelength-index
    delta: width of the window
    CMF: color matching function
    """
    Norm = np.sum(CMF, 0)
    c0 = 0
    c1 = 0
    c2 = 0
    L = CMF.shape[0]
    for j in range(int(i-delta/2), int(i+delta/2), 1):
        if( (j > 0) and (j < L) ):
            c0 += CMF[j,0]
            c1 += CMF[j,1]
            c2 += CMF[j,2]
        elif( j > L ):
            c0 += CMF[j-L,0]
            c1 += CMF[j-L,1]
            c2 += CMF[j-L,2]
        elif( j < 0 ):
            c0 += CMF[L+j,0]
            c1 += CMF[L+j,1]
            c2 += CMF[L+j,2]
    return (c0/Norm[0], c1/Norm[1], c2/Norm[2])

In [16]:
# plot single-sided Newtonian optimal solid 
C0 = []
C1 = []
C2 = []
RGB = []
fig = plt.figure()
ax = fig.gca(projection='3d')
for i in range(0, 441, 10):
    for j in range(0, 441, 20):
        O = EdgeOptimal(i, j, CMF)
        C0.append( O[0] )
        C1.append( O[1] )
        C2.append( O[2] )
        C = XYZColor(O[0], O[1], O[2])
        crgb = convert_color(C, sRGBColor)
        r = crgb.rgb_r
        g = crgb.rgb_g
        b = crgb.rgb_b
        r = r if r > 0 else 0
        g = g if g > 0 else 0
        b = b if b > 0 else 0
        r = r if r < 1 else 1
        g = g if g < 1 else 1
        b = b if b < 1 else 1
        rgb = [r, g, b]
        RGB.append(rgb)
ax.scatter(C0, C1, C2, c=RGB)


In [18]:
# plot double-sided Schroedinger color solid, with locus of windows 1/2 spectral width

C0 = []
C1 = []
C2 = []
RGB = []
fig = plt.figure()
ax = fig.gca(projection='3d')
for i in range(0, 441, 10):
    for j in range(0, 441, 20):
        O = DoubleOptimal(i, j, CMF)
        C0.append( O[0] )
        C1.append( O[1] )
        C2.append( O[2] )
        C = XYZColor(O[0], O[1], O[2])
        crgb = convert_color(C, sRGBColor)
        r = crgb.rgb_r
        g = crgb.rgb_g
        b = crgb.rgb_b
        r = r if r > 0 else 0
        g = g if g > 0 else 0
        b = b if b > 0 else 0
        r = r if r < 1 else 1
        g = g if g < 1 else 1
        b = b if b < 1 else 1
        rgb = [r, g, b]
        RGB.append(rgb)
ax.scatter(C0, C1, C2, c=RGB)

# plot optimal colors with width = L/2
C0 = []
C1 = []
C2 = []
RGB = []
for i in range(0, 441, 10):
    O = DoubleOptimal(i, 220, CMF)
    C0.append( O[0] )
    C1.append( O[1] )
    C2.append( O[2] )
    C = XYZColor(O[0], O[1], O[2])
    crgb = convert_color(C, sRGBColor)
    r = crgb.rgb_r
    g = crgb.rgb_g
    b = crgb.rgb_b
    r = r if r > 0 else 0
    g = g if g > 0 else 0
    b = b if b > 0 else 0
    r = r if r < 1 else 1
    g = g if g < 1 else 1
    b = b if b < 1 else 1
    rgb = [r, g, b]
    RGB.append(rgb)
ax.scatter(C0, C1, C2, s=400, c=RGB) 
plt.show()

Observe that we seem to get an optimal color locus when we plot equal width optimal windows in XYZ. Let's verify this

In [17]:
# Plot all optimal colors of constant width on a line
fig = plt.figure()
ax = fig.gca(projection='3d')
C0 = []
C1 = []
C2 = []
RGB = []
for i in range(0, 441, 10):
    O = DoubleOptimal(i, 220, CMF)
    C0.append( O[0] )
    C1.append( O[1] )
    C2.append( O[2] )
    C = XYZColor(O[0], O[1], O[2])
    crgb = convert_color(C, sRGBColor)
    r = crgb.rgb_r
    g = crgb.rgb_g
    b = crgb.rgb_b
    r = r if r > 0 else 0
    g = g if g > 0 else 0
    b = b if b > 0 else 0
    r = r if r < 1 else 1
    g = g if g < 1 else 1
    b = b if b < 1 else 1
    rgb = [r, g, b]
    ax.scatter(i, 0, s=400, color=rgb) 
plt.show()

To be sure we're doing this right, lets see what happens when we plot with just edge optimals

In [31]:
# Plot only edge optimals of constant width on a line
fig = plt.figure()
ax = fig.gca(projection='3d')
C0 = []
C1 = []
C2 = []
RGB = []
for i in range(0, 441, 10):
    O = EdgeOptimal(i, 220, CMF)
    C0.append( O[0] )
    C1.append( O[1] )
    C2.append( O[2] )
    C = XYZColor(O[0], O[1], O[2])
    crgb = convert_color(C, sRGBColor)
    r = crgb.rgb_r
    g = crgb.rgb_g
    b = crgb.rgb_b
    r = r if r > 0 else 0
    g = g if g > 0 else 0
    b = b if b > 0 else 0
    r = r if r < 1 else 1
    g = g if g < 1 else 1
    b = b if b < 1 else 1
    rgb = [r, g, b]
    ax.scatter(i, 0, s=400, color=rgb) 
plt.show()

In [36]:
def ColorState(rho):
    """
    ColorState: 
    maps a given density operator 
    to a color in optimal color space. 
    
    TODO
    """